In [52]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=92b1029775d074b2d7e98536b9527bf91ba001270e8688f03e7ee7c67304c17c
  Stored in directory: /home/onyxia/.cache/pip/wheels/e4/62/1d/d4d1bc4f33350ff84227f89b258edb552d604138e3739f5c83
Successfully built bs4


In [80]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [54]:
import functions as fc

In [ ]:
url_ffe = 'http://www.echecs.asso.fr/'
suffix_all_clubs = 'ListeClubs.aspx?Action=CLUBCOMITE&ComiteRef='

## Faire liste clubs

In [57]:
def get_infos_club_from_row(row):
    fields = row.find_all('td')
    departement = fields[0].text.strip()
    commune = fields[1].text.strip()
    club = fields[2].text.strip()
    try:
        suffix_club = fields[2].find('a').get('href')
    except:
        suffix_club = None
    return [club, suffix_club, commune, departement]

In [67]:
def get_all_clubs_in_departement(departement):
    
    liste_clubs_departement = []
    
    url = url_ffe + suffix_all_clubs + "{:02d}".format(departement)
    request_text = requests.get(url).text
    soup = BeautifulSoup(request_text, 'html.parser')
    
    try:
        zone = soup.find('div', {'class':'page-mid'})
        tableau = zone.find('table')
        rows = tableau.find_all('tr')

        for i in range(1,len(rows)):
            liste_clubs_departement.append(get_infos_club_from_row(rows[i]))
    except:
        pass
    return liste_clubs_departement

In [75]:
def get_all_clubs_france():
    liste_clubs = []
    for departement in tqdm(range(100)):
        liste_clubs += get_all_clubs_in_departement(departement)
    return liste_clubs

In [76]:
liste_clubs = get_all_clubs_france()

100%|██████████| 100/100 [00:11<00:00,  8.99it/s]


In [83]:
df_clubs = pd.DataFrame(liste_clubs, columns = ['club_name', 'suffix_url', 'commune', 'departement'])

In [84]:
df_clubs

,club_name,suffix_url,commune,departement
0,La Tour de Bage,FicheClub.aspx?Ref=2833,BAGE LA VILLE,01
1,Club d'Echecs de Belley,FicheClub.aspx?Ref=3232,BELLEY,01
2,Cercle d'Echecs Bressan,FicheClub.aspx?Ref=1168,BOURG EN BRESSE,01
3,La Tour de la Dombes,FicheClub.aspx?Ref=1171,CHATILLON SUR CHALARONNE,01
4,Amicale Echecs,FicheClub.aspx?Ref=2725,GEX,01
...,...,...,...,...
812,C.E. Air France Siege Roissy,FicheClub.aspx?Ref=934,ROISSY EN FRANCE,95
813,L'Echiquier de Santeuil,FicheClub.aspx?Ref=2506,SANTEUIL,95
814,L'Echiquéenne de Survilliers,FicheClub.aspx?Ref=2185,SURVILLIERS,95
815,Le Pion Passé Vaurealien,FicheClub.aspx?Ref=2810,VAUREAL,95


## Faire liste joueurs